In [39]:
import billboard
from datetime import timedelta, date
import time

d = {}

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2015, 1, 1)
end_date = date(2016, 7, 1)
fileSong = open("Song.txt", "x")
for single_date in daterange(start_date, end_date):
    if single_date.weekday() == 0:
        day = single_date.strftime("%Y-%m-%d")
        
        while True:
            try:
                chart = billboard.ChartData('hot-100', date=day)
                #chart2 = billboard.ChartData('artist-100', date=day)
                break
            except:
                print('waiting 60 sec...')
                time.sleep(60)
        
        #print(type(chart))
        #print(chart)
        #fileArtist = open(day+"Artist.txt", "x")
        
        for s in chart:
            fileSong = open("Song.txt", "a")
            fileSong.write(s.title + ": " + s.artist + "\n")

FileExistsError: [Errno 17] File exists: 'Song.txt'

In [5]:
file = open("Song.txt", "r")
artistsongs = {}

#this is a for loop going through each line and mapping the artist to the number of songs it has on the list

for line in file:
    line = line.strip()
    larr = line.split(":")
    #print(larr)
    art = larr[1]
    
    art = art[1:]
    if 'featuring' in art.lower():
        index = art.lower().find('featuring')
        art = art[:index-1]
        art.strip()
    song = larr[0]
    if art not in artistsongs:
        artistsongs[art] = set()
    artistsongs[art].add(song)
print(len(artistsongs))        
#print(artistsongs)
#smallart = sorted(artistsongs.keys())[:10]
#print(smallart)
#print(type(artistsongs))

#print(smallart10)

528
['2 Chainz', '2 Chainz x Gucci Mane x Quavo', '21 Savage', '21 Savage & Metro Boomin', '21 Savage, Offset & Metro Boomin', '4', '5 Seconds Of Summer', '6', '6LACK', '6ix9ine']


In [6]:
#sorts the artists by the number of songs they have had on the top 100 set
import operator
artistnum = {}
for k in artistsongs:
    artistnum[k] = len(artistsongs[k])

numsort = sorted(artistnum.items(), key=operator.itemgetter(1), reverse = True)
#print(artistnum)
#print(numsort)

In [7]:
#goes through and creates a dictionary with the number of times the song has been on the top 100
import numpy as np
import matplotlib.pyplot as plt
file = open("Songtest.txt", "r")
songnum = {}
for line in file:
    line = line.strip()
    larr = line.split(":")
    song = larr[0]
    if song not in songnum:
        songnum[song] = 0
    songnum[song] += 1
#print(songnum)
songsort = sorted(songnum.items(), key=operator.itemgetter(1), reverse = True)
top10 = songsort[:10]
print(songsort[:10])
#plt.hist(songsort[:10])

[('Unforgettable', 41), ('Shape Of You', 22), ('Despacito', 21), ("That's What I Like", 21), ('Believer', 21), ("There's Nothing Holdin' Me Back", 21), ('Attention', 21), ('Humble.', 21), ('Bank Account', 21), ('Sorry Not Sorry', 21)]


In [8]:
import re
import urllib.request
from bs4 import BeautifulSoup
 
def get_lyrics(artist,song_title):
    artist = artist.lower()
    song_title = song_title.lower()
  
  # remove all except alphanumeric characters from artist and song_title
    artist = re.sub('[^A-Za-z0-9]+', "", artist)
    song_title = re.sub('[^A-Za-z0-9]+', "", song_title)
    if artist.startswith("the"):    # remove starting 'the' from artist e.g. the who -> who
        artist = artist[3:]
    url = "http://azlyrics.com/lyrics/"+artist+"/"+song_title+".html"
    
    try:
        content = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(content, 'html.parser')
        lyrics = str(soup)
        # lyrics lies between up_partition and down_partition
        up_partition = '<!-- Usage of azlyrics.com content by any third-party lyrics provider is prohibited by our licensing agreement. Sorry about that. -->'
        down_partition = '<!-- MxM banner -->'
        lyrics = lyrics.split(up_partition)[1]
        lyrics = lyrics.split(down_partition)[0]
        lyrics = lyrics.replace('<br>','').replace('</br>','').replace('</div>','').strip()
        return lyrics
    except Exception as e:
        return "Exception occurred \n" +str(e)

In [9]:
import re
import nltk

# Get a list of stopwords from nltk
stopwords = nltk.corpus.stopwords.words("english")

def get_clean_words(lyrics):
    def _isnum(w):
        try:
            int(w)
            return True
        except ValueError:
            return False
        
    # Remove table and external links
    markup_text = re.sub(r'\{\{[\s\S]*?\}\}', '', lyrics)

    # Remove category links
    markup_text = re.sub(r'\[\[Category.+\]\]', '', markup_text)

    # Set words to lowercase and remove them if they are stop words
    words = [w.lower() for w in re.findall('\w+', markup_text) if w.lower() not in stopwords]

    # Remove numbers
    wordswoutnums = [w for w in words if not _isnum(w)]
    
    #remove br
    words = []
    for w in wordswoutnums:
        if w != 'br':
            words.append(w)
        
            
        

    return words

In [11]:
songs = []
dwords = {}
dlen = {}
wordset = set()
#this keeps track of hohw many times the word shows up in the different songs
#print(smallart)
for s in artistsongs:
    #print(s)
    artistsong = artistsongs[s]
    #print(artistsong)
    for item in artistsong:
        #print(item)
        #songs.append(item)
        #print(songs)
        lyrics = get_lyrics(s, item)
        #print(lyrics)
        cleanwords = get_clean_words(lyrics)
        if not (len(cleanwords) == 5 and 'http' in cleanwords):
            songs.append(item)
            #print(cleanwords)
            for cw in cleanwords:
                wordset.add(cw)
            dwords[item] = cleanwords
            dlen[item] = len(cleanwords)
wordset = sorted(wordset)
#print(len(wordset))
#print(wordset)
#print(d)
print(songs)
    

KeyboardInterrupt: 

In [132]:
matrix = []
for song in songs:
    vec = [0] * len(wordset)
    cleanwords = d[song]
    for w in wordset:
        if w in cleanwords:
            vec[wordset.index(w)] += 1
    matrix.append(vec)
print(np.array(matrix))

[[0 0 0 ... 0 1 0]
 [0 0 0 ... 1 0 0]
 [1 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [134]:
lol = np.array(matrix)

TF = lol / lol.sum(1).reshape(-1, 1)

rows = lol.shape[0]
TFIDF = TF * np.log(rows / ((lol > 0).sum(0)))
TFIDF

array([[0.        , 0.        , 0.        , ..., 0.        , 0.01369844,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.02580689, 0.        ,
        0.        ],
       [0.01864756, 0.        , 0.01864756, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [192]:
dsorted = sorted(dlen.items(), key = operator.itemgetter(1), reverse = True)
#print(dsorted)
topbunch = dsorted[:5]
print(topbunch)
song1 = topbunch[0][0]
song2 = topbunch[1][0]
song3 = topbunch[2][0]

song1index1 = songs.index(song1)
#print(song1index1)
#print(song1)
song1index = np.where(songs == song1)
#print(song1index)
#print(type(song1index))
song1blah = sorted(list(zip(TFIDF[song1index1], wordset)), reverse = True)[:10]
            #sorted(list(zip(TFIDF[int(am1index[0])], words)), reverse=True)[:10]
print(song1blah)

[('Bank Account', 437), ('4 AM', 392), ("It's A Vibe", 255), ('Kooda', 247), ('Scars To Your Beautiful', 224)]
2
Bank Account
[(0.01864755972836235, 'wrong'), (0.01864755972836235, 'woes'), (0.01864755972836235, 'whore'), (0.01864755972836235, 'whole'), (0.01864755972836235, 'watchin'), (0.01864755972836235, 'used'), (0.01864755972836235, 'umm'), (0.01864755972836235, 'type'), (0.01864755972836235, 'try'), (0.01864755972836235, 'things')]
